In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FuncFormatter
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score
#from sklearn.preprocessing import LabelEncoder
#from sklearn.metrics import mean_squared_error

In [2]:
#load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
#exploring data
train.head()

,id,worker_group_code,worker_group_name,department_code,department_name,union_code,union_name,job_group_code,job_group,job_code,job,salary
0,1,4,Community Health,DPH,DPH Public Health,791.0,"SEIU - Staff and Per Diem Nurses, Local 1021",2300,Nursing,2328,Nurse Practitioner,2139.13
1,2,2,"Public Works, Transportation & Commerce",DPW,DPW GSA - Public Works,21.0,"Prof & Tech Engineers - Miscellaneous, Local 21",5300,Sub-Professional Engineering,5380,"Stdntdsgntrain1, Arch/Eng/Plng",8769.90
2,3,6,General Administration & Finance,TIS,DT GSA - Technology,790.0,"SEIU - Miscellaneous, Local 1021",1600,"Payroll, Billing & Accounting",1634,Principal Account Clerk,27267.91
3,4,4,Community Health,DPH,DPH Public Health,351.0,Municipal Executive Association - Miscellaneous,900,Management,2620,Food Service Mgr Administrator,396.58
4,5,2,"Public Works, Transportation & Commerce",DPW,DPW GSA - Public Works,21.0,"Prof & Tech Engineers - Miscellaneous, Local 21",6300,Construction Inspection,6318,Construction Inspector,27519.82


In [4]:
train.dtypes

id                     int64
worker_group_code      int64
worker_group_name     object
department_code       object
department_name       object
union_code           float64
union_name            object
job_group_code        object
job_group             object
job_code              object
job                   object
salary               float64
dtype: object

In [5]:
#converting the object datatypes to numerical as the model takes only numerical 
#train dataset
train['job_group_code'] = pd.to_numeric(train['job_group_code'], errors='coerce')
train['job_code'] = pd.to_numeric(train['job_code'], errors = 'coerce')

#test dataset
test['job_group_code'] = pd.to_numeric(test['job_group_code'], errors='coerce')
test['job_code'] = pd.to_numeric(test['job_code'], errors = 'coerce')



In [6]:
#fill the NaN with 0s
#train dataset
train['job_group_code'].fillna(0, inplace = True) 
train['job_code'].fillna(0, inplace = True)
train['union_code'].fillna(0, inplace = True)
train['salary'].fillna(0, inplace = True)
#test dataset
test['job_group_code'].fillna(0, inplace = True) 
test['job_code'].fillna(0, inplace = True)
test['union_code'].fillna(0, inplace = True)


In [7]:
#encoding the department_code 
dept_mapping = {'AAM': 1, 'ADM': 2, 'ADP': 3, 'AIR': 4, 'ART': 5, 'ASR': 6, 'BOS': 7, 'CAT': 8, 'CFC': 9, 'CHF': 10,
          'CLP': 11, 'CON': 12, 'CPC': 13, 'CRT': 14, 'CSC': 15, 'CSS': 16, 'CWP': 17, 'DAT': 18, 'DBI': 19, 
         'DPH': 20, 'DPW': 21, 'DSS': 22, 'ECD': 23, 'ECN': 24, 'ENV': 25, 'ETH': 26, 'FAM': 27, 'FIR': 28, 'HHP': 29, 
          'HOM': 30, 'HRC': 31, 'HRD': 32, 'HSS': 33, 'JUV': 34, 'LIB': 35, 'LLB': 36, 'MTA': 37, 'MYR' : 38, 'PAB': 39, 
         'PDR' : 40, 'POL': 41, 'PRT': 42, 'PUC': 43, 'REC': 44, 'REG' : 45, 'RET': 46, 'RNT': 47, 'SCI': 48, 'SHF': 49, 
         'TIS': 50, 'TTX': 51, 'UNA': 52, 'WAR': 53, 'WOM': 54, 'WTR': 55}
train.replace({'department_code': dept_mapping}, inplace = True)


In [8]:
features = ['worker_group_code', 'department_code', 'union_code', 'job_group_code', 'job_code']
target = ['salary']
X_train = train[features]
Y_train = train[target]
X_test = test[features]

In [9]:
#feature selection
fsel = SelectKBest(score_func=f_regression, k=1)
fit = fsel.fit(X_train, Y_train)
print(fit.scores_)

[ 1103.96183707    30.35356493   314.809997     867.13657923   678.56080443]


C:\Users\arpit\Anaconda2\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [10]:
#dummy encoding on the feature with the highest score
dummy1_train = pd.get_dummies(X_train['worker_group_code'], prefix = 'worker_group')
dummy1_test = pd.get_dummies(X_test['worker_group_code'], prefix = 'worker_group')

#drop the unnecessary features
dummy1_train = dummy1_train.drop('worker_group_7', 1)
dummy1_train = dummy1_train.drop('worker_group_6', 1)
dummy1_train = dummy1_train.drop('worker_group_5', 1)
dummy1_train = dummy1_train.drop('worker_group_4', 1)
dummy1_train = dummy1_train.drop('worker_group_3', 1)

dummy1_test = dummy1_test.drop('worker_group_7', 1)
dummy1_test = dummy1_test.drop('worker_group_6', 1)
dummy1_test = dummy1_test.drop('worker_group_5', 1)
dummy1_test = dummy1_test.drop('worker_group_4', 1)
dummy1_test = dummy1_test.drop('worker_group_3', 1)



In [11]:
lr = LinearRegression(fit_intercept = False, normalize = True)
lr.fit(dummy1_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=1, normalize=True)

In [12]:
test_predict = lr.predict(dummy1_test)
print(test_predict)

[[ 88166.91090766]
 [ 88166.91090766]
 [ 67637.13115348]
 ..., 
 [     0.        ]
 [ 67637.13115348]
 [ 67637.13115348]]


In [13]:
lr.score(dummy1_train, Y_train)

-0.69365141416097464

In [14]:
print(lr.coef_)

[[ 88166.91090766  67637.13115348]]


In [16]:
lr.score(dummy1_test, test_predict)

1.0

In [ ]:
prediction=pd.DataFrame(test_predict, columns=['pred']) 
result = pd.DataFrame({'id': test['id'], 'salary': prediction['pred']})
result.to_csv('result.csv', index = False)